In [1]:
import os
print(os.path.exists(r"C:\Windows\Fonts\simsun.ttc"))


True


In [2]:

# 执行以下代码，无报错则说明安装成功
import kaleido
import plotly.graph_objects as go
print("kaleido 版本：", kaleido.__version__)
print("plotly 版本：", go.__version__)

ModuleNotFoundError: No module named 'kaleido'

In [ ]:
from qgis.core import (
    QgsVectorLayer, QgsFeature, QgsGeometry, QgsPointXY,
    QgsField, QgsProject, QgsSymbolLayer, QgsLineSymbol,
    QgsSingleSymbolRenderer, QgsArrowSymbolLayer, QgsUnitTypes
)
from qgis.PyQt.QtCore import QVariant
from qgis.PyQt.QtGui import QColor  # 补充缺失的QColor导入（之前代码漏了）
from qgis.core import QgsWkbTypes  # 补充WkbTypes导入

# -------------------------- 配置参数（根据你的图层修改） --------------------------
INPUT_LAYER_NAME = "01_cluster_工作日_17_for_qgis"  # 替换为你的点图层名（在QGIS图层列表里的名称）
OUTPUT_LAYER_NAME = "01_cluster_工作日_17_lines"  # 生成的线图层名称
FLOW_FIELD = "flow"  # 控制线粗细的字段名（替换为你的实际字段名）
# 线宽缩放系数（根据flow值大小调整，比如flow是1-100，系数设0.1）
LINE_WIDTH_SCALE = 0.1

# -------------------------- 1. 读取输入点图层 --------------------------
# 获取QGIS当前项目中的点图层
input_layer = QgsProject.instance().mapLayersByName(INPUT_LAYER_NAME)[0]
if not input_layer or input_layer.geometryType() != QgsWkbTypes.PointGeometry:
    raise ValueError("未找到指定的点图层，或图层类型不是点！")

# -------------------------- 2. 创建输出线图层 --------------------------
# 定义线图层的属性（继承原图层的cluster_id、flow等字段）
fields = input_layer.fields()
# 创建内存线图层
output_layer = QgsVectorLayer("LineString?crs=" + input_layer.crs().authid(),
                              OUTPUT_LAYER_NAME, "memory")
provider = output_layer.dataProvider()
# 添加属性字段
provider.addAttributes(fields)
output_layer.updateFields()

# -------------------------- 3. 遍历点要素，生成线 --------------------------
features = []
for feature in input_layer.getFeatures():
    # 获取属性中的坐标和flow值
    ox = feature["ox"]
    oy = feature["oy"]
    dx = feature["dy"]  # 注意确认字段名：是dy还是dy？
    dy = feature["dy"]
    flow = feature[FLOW_FIELD] if FLOW_FIELD in feature.fields().names() else 1
    
    # 跳过空值
    if not (ox and oy and dx and dy):
        continue
    
    # 创建起点和终点
    start_point = QgsPointXY(ox, oy)
    end_point = QgsPointXY(dx, dy)
    # 创建线几何
    line_geom = QgsGeometry.fromPolylineXY([start_point, end_point])
    
    # 创建线要素，继承原属性
    line_feature = QgsFeature()
    line_feature.setGeometry(line_geom)
    line_feature.setAttributes(feature.attributes())
    features.append(line_feature)

# 将线要素添加到输出图层
provider.addFeatures(features)
output_layer.updateExtents()

# -------------------------- 4. 设置线样式：宽度绑定flow + 流向箭头 --------------------------
# 创建线符号（基础线）
line_symbol = QgsLineSymbol.createSimple({})
# 清除默认符号层，重新构建
line_symbol.deleteSymbolLayer(0)

# 1. 添加基础线层（宽度由flow控制）
base_line_layer = QgsSimpleLineSymbolLayer()
# 设置线宽绑定字段：flow值 * 缩放系数，单位为毫米
base_line_layer.setDataDefinedProperty(
    QgsSymbolLayer.PropertyWidth,
    QgsProperty.fromExpression(f"{FLOW_FIELD} * {LINE_WIDTH_SCALE}")
)
base_line_layer.setColor(QColor(0, 100, 200))  # 线的颜色（蓝）
base_line_layer.setPenStyle(Qt.SolidLine)
line_symbol.appendSymbolLayer(base_line_layer)

# 2. 添加箭头层（流向终点）
arrow_layer = QgsArrowSymbolLayer()
arrow_layer.setArrowType(QgsArrowSymbolLayer.ArrowSingle)  # 单箭头
arrow_layer.setHeadWidth(0.5)  # 箭头宽度（毫米）
arrow_layer.setHeadLength(1.0)  # 箭头长度（毫米）
arrow_layer.setPlacement(QgsArrowSymbolLayer.ArrowAtEnd)  # 箭头在终点
arrow_layer.setColor(QColor(255, 0, 0))  # 箭头颜色（红）
line_symbol.appendSymbolLayer(arrow_layer)

# 应用符号到图层
renderer = QgsSingleSymbolRenderer(line_symbol)
output_layer.setRenderer(renderer)

# -------------------------- 5. 将线图层加载到QGIS界面 --------------------------
QgsProject.instance().addMapLayer(output_layer)

# 替换原来的f-string
print("✅ 成功生成线图层！共生成 %d 条带箭头的线" % len(features))